In [10]:
%pip install pyarrow
%pip install pandas
%pip install openai
%pip install dotenv


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: openai in d:\installation_folder\python311\lib\site-packages (1.99.9)




[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for dotenv from https://files.pythonhosted.org/packages/b2/b7/545d2c10c1fc15e48653c91efde329a790f2eecfbbf2bd16003b5db2bab0/dotenv-0.9.9-py2.py3-none-any.whl.metadata
  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/5f/ed/539768cf28c661b5b068d66d96a2f155c4971a5d55684a514c1a0e0dec2f/python_dotenv-1.1.1-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pyarrow.parquet as pq
import pandas as pd

file_path = 'prompts_full_dataset.parquet'
parquet_file = pq.ParquetFile(file_path)

schema = parquet_file.schema
unique_columns = []
seen = set()
for field in schema:
    if field.name not in seen:
        unique_columns.append(field.name)
        seen.add(field.name)


table = parquet_file.read(columns=unique_columns)
df = table.to_pandas()

print("Columns in the dataset:", df.columns)
print("\nFirst few rows of the dataset:")
print(df.loc[0, 'prompt'])

Columns in the dataset: Index(['project', 'commit_id', 'func', 'vul', 'CVE ID', 'CWE ID', 'CWE Name',
       'CWE Description', 'Potential Mitigation', 'None', 'None', 'prompt',
       'prompt_char_count', 'prompt_token_count'],
      dtype='object')

First few rows of the dataset:
You are a professional cybersecurity analyst with expertise in static code analysis and Common Weakness Enumeration (CWE) classification.

You will be provided with:
- One **input function**, whose vulnerability status you must assess.
- Several **reference examples**, each containing:
  - A code function
  - A known vulnerability label (1 for vulnerable, 0 for not)
  - CWE ID (if vulnerable)
  - CWE Name (if vulnerable)

---

### TASK

Analyze the **structure, logic, and behavior** of the **input function**. Use deep comparison and reasoning based on the structure, intent, and usage patterns in the reference examples. Focus on the logical flow, operations performed, and overall behavior of the code to asses

In [3]:
print("Sum of tokens in all prompts:", df['prompt_token_count'].sum())

Sum of tokens in all prompts: 3948621


In [4]:
unique_cwe_ids = df['CWE ID'].nunique()
print(f"Number of unique CWE IDs: {unique_cwe_ids}")

num_functions = df['func'].nunique()
print(f"Number of unique functions: {num_functions}")

num_prompts = df['prompt'].nunique()
print(f"Number of unique prompts: {num_prompts}")

avg_char_count = df['prompt_char_count'].mean()
avg_token_count = df['prompt_token_count'].mean()
print(f"Average character count of prompts: {avg_char_count:.2f}")
print(f"Average token count of prompts: {avg_token_count:.2f}")

unique_projects = df['project'].nunique()
print(f"Number of unique projects: {unique_projects}")

Number of unique CWE IDs: 13
Number of unique functions: 2253
Number of unique prompts: 2253
Average character count of prompts: 7011.89
Average token count of prompts: 1752.61
Number of unique projects: 377


In [5]:
def check_func_in_prompt(row):
    func = row['func']
    prompt = row['prompt']
    if isinstance(func, str) and isinstance(prompt, str):
        return func in prompt
    return False

df['func_in_prompt'] = df.apply(check_func_in_prompt, axis=1)

print("\nSample of functions and their presence in prompts:")
print(df[['func', 'prompt', 'func_in_prompt']].head(10))

num_funcs_in_prompt = df['func_in_prompt'].sum()
print(f"\nNumber of functions present in their corresponding prompts: {num_funcs_in_prompt}")
print(f"Number of functions NOT present in their corresponding prompts: {len(df) - num_funcs_in_prompt}")


Sample of functions and their presence in prompts:
                                                func  \
0  FLAC__bool read_residual_partitioned_rice_(FLA...   
1  NeXTPreDecode(TIFF* tif, uint16 s)\n{\n\tstati...   
2  int ncp_open_create_file_or_subdir(struct ncp_...   
3  static MagickBooleanType IsWEBPImageLossless(c...   
4  static Image *ReadTIFFImage(const ImageInfo *i...   
5  bool ImageBitmap::isAccelerated() const {\n  r...   
6  int vfs_fallocate(struct file *file, int mode,...   
7  HTMLElement& toHTMLElement(FormAssociatedEleme...   
8  static void download_one_url(const char *url)\...   
9  static av_cold int vqa_decode_init(AVCodecCont...   

                                              prompt  func_in_prompt  
0  You are a professional cybersecurity analyst w...            True  
1  You are a professional cybersecurity analyst w...            True  
2  You are a professional cybersecurity analyst w...            True  
3  You are a professional cybersecurity analyst

In [ ]:
print(df.loc[0, 'prompt'])

0


In [6]:
def check_func_in_prompt(row):
    func = row['func']
    prompt = row['prompt']
    if isinstance(func, str) and isinstance(prompt, str):
        return func in prompt
    return False

In [7]:
df['func_in_prompt'] = df.apply(check_func_in_prompt, axis=1)

print("\nSample of functions and their presence in prompts (first 5 rows):")
print(df[['func', 'prompt', 'func_in_prompt']].head(5))

# Inspect rows where func is NOT in prompt
print("\nSample of rows where function is NOT present in prompt:")
not_present = df[~df['func_in_prompt']][['func', 'prompt']].head(5)
for idx, row in not_present.iterrows():
    print(f"\nRow {idx}:")
    print(f"Function: {row['func']}")
    print(f"Prompt: {row['prompt']}")

num_funcs_in_prompt = df['func_in_prompt'].sum()
num_funcs_not_in_prompt = len(df) - num_funcs_in_prompt
print(f"\nNumber of functions present in prompts: {num_funcs_in_prompt}")
print(f"Number of functions NOT present in prompts: {num_funcs_not_in_prompt}")

print(f"\nMissing func values: {df['func'].isna().sum()}")
print(f"Missing prompt values: {df['prompt'].isna().sum()}")


Sample of functions and their presence in prompts (first 5 rows):
                                                func  \
0  FLAC__bool read_residual_partitioned_rice_(FLA...   
1  NeXTPreDecode(TIFF* tif, uint16 s)\n{\n\tstati...   
2  int ncp_open_create_file_or_subdir(struct ncp_...   
3  static MagickBooleanType IsWEBPImageLossless(c...   
4  static Image *ReadTIFFImage(const ImageInfo *i...   

                                              prompt  func_in_prompt  
0  You are a professional cybersecurity analyst w...            True  
1  You are a professional cybersecurity analyst w...            True  
2  You are a professional cybersecurity analyst w...            True  
3  You are a professional cybersecurity analyst w...            True  
4  You are a professional cybersecurity analyst w...            True  

Sample of rows where function is NOT present in prompt:

Row 28:
Function:  static void copyMono8(
         short *dst,
        const int *const *src,
         unsigned 

In [8]:
def check_func_in_prompt_normalized(row):
    func = row['func']
    prompt = row['prompt']
    if isinstance(func, str) and isinstance(prompt, str):
        func = ' '.join(func.split())
        prompt = ' '.join(prompt.split())
        return func in prompt
    return False

df['func_in_prompt_normalized'] = df.apply(check_func_in_prompt_normalized, axis=1)
print("\nWith normalized text:")
print(f"Number of functions present in prompts: {df['func_in_prompt_normalized'].sum()}")
print(f"Number of functions NOT present in prompts: {len(df) - df['func_in_prompt_normalized'].sum()}")


With normalized text:
Number of functions present in prompts: 2253
Number of functions NOT present in prompts: 0


In [21]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import re
load_dotenv()
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def get_vul_info(prompt):
    try:
        response = client.responses.create(
            model="gpt-4o-mini",
            input=prompt,
        )
        text = response.output[0].content[0].text
        
        vul_match = re.search(r"Vulnerability:\s*(.*)", text)
        cwe_id_match = re.search(r"CWE ID:\s*(.*)", text)
        cwe_name_match = re.search(r"CWE Name:\s*(.*)", text)
        
        return (
            text.strip(),
            vul_match.group(1).strip() if vul_match else None,
            cwe_id_match.group(1).strip() if cwe_id_match else None,
            cwe_name_match.group(1).strip() if cwe_name_match else None
        )

    except Exception as e:
        print(f"Error: {e}")

In [24]:
gpt4oMiniResponseDF = df.copy()
gpt4oMiniResponseDF["response"] = None
gpt4oMiniResponseDF["vul"] = None
gpt4oMiniResponseDF["CWE ID"] = None
gpt4oMiniResponseDF["CWE Name"] = None
gpt4oMiniResponseDF["CWE Description"] = None
gpt4oMiniResponseDF["CVE ID"] = None
gpt4oMiniResponseDF["Potential Mitigation"] = None
print(gpt4oMiniResponseDF.head())

       project                                 commit_id  \
0         flac  fcf0ba06ae12ccd7c67cee3c8d948df15f946b85   
1      libtiff  b18012dae552f85dcc5c57d3bf4e997a15b1cc1c   
2      staging  4c41aa24baa4ed338241d05494f2c595c885af8f   
3  ImageMagick  361ed689cc8e56fd125f9d0d6508e9eb303bdca6   
4  ImageMagick  986b5dff173413fa712db27eb677cdef15f0bab6   

                                                func   vul CVE ID CWE ID  \
0  FLAC__bool read_residual_partitioned_rice_(FLA...  None   None   None   
1  NeXTPreDecode(TIFF* tif, uint16 s)\n{\n\tstati...  None   None   None   
2  int ncp_open_create_file_or_subdir(struct ncp_...  None   None   None   
3  static MagickBooleanType IsWEBPImageLossless(c...  None   None   None   
4  static Image *ReadTIFFImage(const ImageInfo *i...  None   None   None   

  CWE Name CWE Description Potential Mitigation   None  None  \
0     None            None                 None  23682     0   
1     None            None                 None  21469

In [26]:
gpt4oMiniResponseDF[["response", "vul", "CWE ID", "CWE Name"]] = gpt4oMiniResponseDF["prompt"].apply(
    lambda p: pd.Series(get_vul_info(p))
)

In [30]:
cols_to_drop = [
    col for col in gpt4oMiniResponseDF.columns
    if (col is None)
    or (isinstance(col, str) and col.strip().lower() in {"none", "noe"})
]

if not cols_to_drop:
    print("No matching columns to drop.")
else:
    print("Dropping columns:", cols_to_drop)
    gpt4oMiniResponseDF = gpt4oMiniResponseDF.drop(columns=cols_to_drop)

print("\nRemaining columns:")
print(list(gpt4oMiniResponseDF.columns))
print("\nFirst row preview:")
print(gpt4oMiniResponseDF.head(1))

gpt4oMiniResponseDF.to_parquet("gpt_40_mini_results.parquet", index=True)

Dropping columns: ['None', 'None']

Remaining columns:
['project', 'commit_id', 'func', 'vul', 'CVE ID', 'CWE ID', 'CWE Name', 'CWE Description', 'Potential Mitigation', 'prompt', 'prompt_char_count', 'prompt_token_count', 'func_in_prompt', 'func_in_prompt_normalized', 'response']

First row preview:
  project                                 commit_id  \
0    flac  fcf0ba06ae12ccd7c67cee3c8d948df15f946b85   

                                                func vul CVE ID   CWE ID  \
0  FLAC__bool read_residual_partitioned_rice_(FLA...   1   None  CWE-787   

              CWE Name CWE Description Potential Mitigation  \
0  Out-of-bounds Write            None                 None   

                                              prompt  prompt_char_count  \
0  You are a professional cybersecurity analyst w...               7976   

   prompt_token_count  func_in_prompt  func_in_prompt_normalized  \
0                1994            True                       True   

                   

In [ ]:

try:
    gpt4oMiniResponseDF 
except NameError:
    raise NameError("gpt4oMiniResponseDF not found. Load your predictions into 'gpt4oMiniResponseDF' first.")

if "vul" not in gpt4oMiniResponseDF.columns:
    raise KeyError("gpt4oMiniResponseDF must contain a 'vul' column (predictions 0/1).")

if "vul" not in df.columns:
    raise KeyError("original_df must contain a 'vul' column (ground truth 0/1).")

pred = gpt4oMiniResponseDF["vul"]
true = df["vul"]

common_idx = pred.index.intersection(true.index)
if len(common_idx) == 0:
    raise ValueError("No overlapping indices found between gpt4oMiniResponseDF and df. "
                     "Ensure both DataFrames index the same rows, or reindex them appropriately.")

pred = pred.loc[common_idx].copy()
true = true.loc[common_idx].copy()

def to_binary_series(s, colname="vul"):
    s_conv = pd.to_numeric(s, errors="coerce") 
    mask_na = s_conv.isna()
    if mask_na.any():
        s_str = s.astype(str).str.strip().str.lower()
        s_conv.loc[mask_na & s_str.isin({"1", "true", "yes", "y", "t"})] = 1
        s_conv.loc[mask_na & s_str.isin({"0", "false", "no", "n", "f"})] = 0
    s_conv = s_conv.where(s_conv.isin([0, 1]))
    return s_conv

pred_bin = to_binary_series(pred, "predicted_vul")
true_bin = to_binary_series(true, "true_vul")

invalid_mask = pred_bin.isna() | true_bin.isna()
n_invalid = invalid_mask.sum()
if n_invalid > 0:
    print(f"Dropping {n_invalid} row(s) due to non-binary or missing values in 'vul' columns.")
pred_bin = pred_bin[~invalid_mask].astype(int)
true_bin = true_bin[~invalid_mask].astype(int)

TP = int(((pred_bin == 1) & (true_bin == 1)).sum())
FP = int(((pred_bin == 1) & (true_bin == 0)).sum())
FN = int(((pred_bin == 0) & (true_bin == 1)).sum())
TN = int(((pred_bin == 0) & (true_bin == 0)).sum())
total = TP + FP + FN + TN

def safe_div(n, d):
    return float(n) / d if d != 0 else 0.0

precision = safe_div(TP, TP + FP)
recall = safe_div(TP, TP + FN)
accuracy = safe_div(TP + TN, total)
f1 = safe_div(2 * precision * recall, precision + recall) if (precision + recall) > 0 else 0.0

metrics_df = pd.DataFrame([{
    "TP": TP, "FP": FP, "FN": FN, "TN": TN,
    "precision": precision, "recall": recall, "accuracy": accuracy, "f1": f1,
    "total_evaluated_rows": total, "dropped_rows": int(n_invalid)
}], index=["vul(0/1)"])

display_df = metrics_df.copy()
display_df[["precision", "recall", "accuracy", "f1"]] = display_df[["precision", "recall", "accuracy", "f1"]].applymap(lambda x: f"{x:.2%}")
display_df[["TP","FP","FN","TN","total_evaluated_rows","dropped_rows"]] = display_df[["TP","FP","FN","TN","total_evaluated_rows","dropped_rows"]].astype(int)

print("\nBinary classification metrics (based on 'vul' 0/1):\n")
print(display_df.T)

print("\nConfusion matrix counts:")
print(f"TP: {TP}, FP: {FP}, FN: {FN}, TN: {TN}")
print(f"Total evaluated rows: {total}; Rows dropped (invalid/missing): {int(n_invalid)}")



Binary classification metrics (based on 'vul' 0/1):

                     vul(0/1)
TP                        698
FP                        839
FN                        175
TN                        541
precision              45.41%
recall                 79.95%
accuracy               54.99%
f1                     57.93%
total_evaluated_rows     2253
dropped_rows                0

Confusion matrix counts:
TP: 698, FP: 839, FN: 175, TN: 541
Total evaluated rows: 2253; Rows dropped (invalid/missing): 0


C:\Users\USER\AppData\Local\Temp\ipykernel_13240\1732693473.py:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  display_df[["precision", "recall", "accuracy", "f1"]] = display_df[["precision", "recall", "accuracy", "f1"]].applymap(lambda x: f"{x:.2%}")


In [33]:
try:
    gpt4oMiniResponseDF
except NameError:
    raise NameError("gpt4oMiniResponseDF not found. Load your predictions into 'gpt4oMiniResponseDF' first.")

try:
    df
except NameError:
    raise NameError("Ground truth DataFrame 'df' not found. Load it into variable 'df' (must contain 'vul' and 'CWE ID').")

for col in ["vul"]:
    if col not in gpt4oMiniResponseDF.columns:
        raise KeyError(f"gpt4oMiniResponseDF must contain '{col}' column.")
    if col not in df.columns:
        raise KeyError(f"df must contain '{col}' column.")

for col in ["CWE ID"]:
    if col not in gpt4oMiniResponseDF.columns:
        raise KeyError(f"gpt4oMiniResponseDF must contain '{col}' column.")
    if col not in df.columns:
        raise KeyError(f"df must contain '{col}' column.")

pred_vul = gpt4oMiniResponseDF["vul"]
true_vul = df["vul"]
common_idx = pred_vul.index.intersection(true_vul.index)
if len(common_idx) == 0:
    raise ValueError("No overlapping indices between prediction and ground-truth DataFrames.")

pred_cwe_raw = gpt4oMiniResponseDF.loc[common_idx, "CWE ID"].copy()
true_cwe_raw = df.loc[common_idx, "CWE ID"].copy()

pred_vul = pred_vul.loc[common_idx].copy()
true_vul = true_vul.loc[common_idx].copy()

def to_binary_series(s):
    s_conv = pd.to_numeric(s, errors="coerce")
    mask_na = s_conv.isna()
    if mask_na.any():
        s_str = s.astype(str).str.strip().str.lower()
        s_conv.loc[mask_na & s_str.isin({"1", "true", "yes", "y", "t"})] = 1
        s_conv.loc[mask_na & s_str.isin({"0", "false", "no", "n", "f"})] = 0
    s_conv = s_conv.where(s_conv.isin([0, 1]))
    return s_conv

pred_bin = to_binary_series(pred_vul)
true_bin = to_binary_series(true_vul)

invalid_mask = pred_bin.isna() | true_bin.isna()
n_invalid = int(invalid_mask.sum())
if n_invalid > 0:
    print(f"Dropping {n_invalid} row(s) due to invalid/missing 'vul' values.")
valid_idx = pred_bin[~invalid_mask].index
pred_bin = pred_bin.loc[valid_idx].astype(int)
true_bin = true_bin.loc[valid_idx].astype(int)
pred_cwe_raw = pred_cwe_raw.loc[valid_idx]
true_cwe_raw = true_cwe_raw.loc[valid_idx]

def normalize_cwe(x):
    if pd.isna(x):
        return ""
    s = str(x).strip().lower()
    s = s.replace("cwe-", "").replace("cwe ", "").strip()
    m = re.search(r"(\d+)", s)
    if m:
        return m.group(1)
    return s

pred_cwe = pred_cwe_raw.apply(normalize_cwe)
true_cwe = true_cwe_raw.apply(normalize_cwe)

true_has_cwe = true_cwe != ""
pred_has_cwe = pred_cwe != ""
cwe_exact_match = (pred_cwe == true_cwe) & true_has_cwe

total_with_true_cwe = int(true_has_cwe.sum())
correct_cwe_all = int(cwe_exact_match.sum())
cwe_accuracy_all = (correct_cwe_all / total_with_true_cwe) if total_with_true_cwe else 0.0

tp_mask = (pred_bin == 1) & (true_bin == 1)
tp_with_true_cwe = int((tp_mask & true_has_cwe).sum())
correct_cwe_in_tp = int((tp_mask & cwe_exact_match).sum())
cwe_accuracy_in_tp = (correct_cwe_in_tp / tp_with_true_cwe) if tp_with_true_cwe else 0.0

predpos_mask = (pred_bin == 1)
predpos_with_true_cwe = int((predpos_mask & true_has_cwe).sum())
correct_cwe_in_predpos = int((predpos_mask & cwe_exact_match).sum())
cwe_accuracy_in_predpos = (correct_cwe_in_predpos / predpos_with_true_cwe) if predpos_with_true_cwe else 0.0

predicted_cwe_count = int(pred_has_cwe.sum())

rows = []
rows.append({
    "metric": "CWE accuracy (among rows with GT CWE)",
    "correct": correct_cwe_all,
    "total": total_with_true_cwe,
    "pct": f"{cwe_accuracy_all:.2%}"
})
rows.append({
    "metric": "CWE accuracy among TP (pred=1 & true=1)",
    "correct": correct_cwe_in_tp,
    "total": tp_with_true_cwe,
    "pct": f"{cwe_accuracy_in_tp:.2%}"
})
rows.append({
    "metric": "CWE accuracy among predicted positives (pred=1)",
    "correct": correct_cwe_in_predpos,
    "total": predpos_with_true_cwe,
    "pct": f"{cwe_accuracy_in_predpos:.2%}"
})
rows.append({
    "metric": "Predicted non-empty CWE count",
    "correct": predicted_cwe_count,
    "total": int(len(valid_idx)),
    "pct": f"{predicted_cwe_count/len(valid_idx):.2%}"
})

cwe_metrics_df = pd.DataFrame(rows).set_index("metric")
print("\nCWE correctness summary:\n")
print(cwe_metrics_df)



CWE correctness summary:

                                                 correct  total      pct
metric                                                                  
CWE accuracy (among rows with GT CWE)                917   2253   40.70%
CWE accuracy among TP (pred=1 & true=1)              376    698   53.87%
CWE accuracy among predicted positives (pred=1)      376   1537   24.46%
Predicted non-empty CWE count                       2253   2253  100.00%
